In [ ]:
from EIS import estimate_impedance, EIS
import numpy as np

help(estimate_impedance)
help(EIS)

In [ ]:
# Specify COM port to which teensy is attached
COM = 'COM9'  # /dev/ttyACM0 (Linux)

# ... and start an EIS instance 
myEIS = EIS(COM) 

print(myEIS.serial.name)

In [ ]:
'''
Execute the measurements and save the results

Here a correct frequency range (f_range) should be picked, for this you need 
to analyse the circuit to find some ballpark figures.

Warning: frequencies used should adhere to the Shannon-Nyquist sampling theorem 
and they should respect the limitations of the acquisition electronics.

Known problem: sometimes a growing list of dots is shown and no further progress. 
If that happens the workaround is to interrupt the kernel and run this cell again.
''' 

f_max, f_step = 5001, 1000  # f_min is set to 0 below

space_style = ['linear', 'logarithmic'][0]

if space_style == 'linear':
    f_range = np.arange(0, f_max, f_step)
elif space_style == 'logarithmic':
    f_range = np.logspace(np.log10(1), np.log10(f_max), 6) # log(0) is not defined
    f_range[0]= 0 # DC_value is important so put it back in
elif 'f_range' not in locals():
    raise ValueError('f_range not initialized')

spectrum = myEIS.measure_spectrum(f_range, 
                                  f_sampling = 25000, 
                                  Rs= 1960,)

print(spectrum)

np.savez('spectrum__Rs1960Ohm.npz',spectrum=spectrum, f_range=f_range)

In [ ]:
'''
Exposing some of the lower level functions used in te measurement below.
Gives a peak into the last executed measurement on the teensy.
'''
stimulus_parameters = myEIS.get_stimulus_parameters()  
print('------------stimulus_parameters----------------')
print(stimulus_parameters) 
print('-----------------------------------------------')
print(myEIS.get_data_slice(1, 0))
print('-----------------------------------------------')
print(myEIS.get_data_slice(2, 100))
print('-----------------------------------------------')
print(myEIS.get_data_slice(3, 2001))  
# sleep(0.1)
print('-----------------------------------------------')
print(myEIS.get_and_print_responses())
V1 = myEIS.get_data(stimulus_parameters, 1)
V2 = myEIS.get_data(stimulus_parameters, 2)
DAC = myEIS.get_data(stimulus_parameters, 3)


%pylab

plt.figure()
#plt.plot(np.array(V1),np.array(V2),'g.')
plt.subplot(2,1,1)
plt.plot(np.array(V1),'go-')
plt.subplot(2,1,2)
plt.plot(np.array(V2),'ro-')
#plt.plot(8*np.array(DAC), 'k.')

In [ ]:
'''
Load earlier acquired data from file
'''

reload_data = False
if reload_data:
    spectrum_storage=np.load('spectrum__Rs1960Ohm.npz')
    spectrum = spectrum_storage['spectrum']
    f_range=spectrum_storage['f_range']

In [ ]:
'''
Create plots of the measured data and a theoretical one for a known circuit. 

This way you can compare measurements on a known testcircuit with theory.

Todo: put info on axes.
'''

%pylab inline

reals = [np.real(pars[0]) for pars in spectrum]
imaginaries = [np.imag(pars[0]) for pars in spectrum]
frequencies = f_range

R_testcircuit, C_testcircuit = 1000, 160*1e-9  # R in Ohm, C in Farad
Zomega = 1000/(1+1j*2*pi*frequencies*R_testcircuit*C_testcircuit)

# print(Zomega)
Zreals = [np.real(Z) for Z in Zomega]
Zimaginaries = [np.imag(Z) for Z in Zomega]

plt.figure()
plt.subplot(3,1,1)
plt.plot(reals, imaginaries,'.')

plt.subplot(3,1,2)
plt.plot(frequencies, reals,'.')

plt.subplot(3,1,3)
plt.plot(frequencies, imaginaries,'.')

plt.figure()
plt.subplot(3,1,2)
plt.plot(frequencies, Zreals, '.')

plt.subplot(3,1,3)
plt.plot(frequencies, Zimaginaries,'.')

plt.subplot(3,1,1)
plt.plot(Zreals, Zimaginaries,'.')


plt.figure()
plt.subplot(1,2,1)
plt.plot(Zreals, reals, '.')


plt.subplot(1,2,2)
plt.plot(Zimaginaries, imaginaries,'.')

